In [14]:
import numpy as np
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [15]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2687
Dimensão vocabulario positivo: 2326


In [16]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)

lexicon = list(vocab_neg.union(vocab_pos))

In [13]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = True))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.816901408451
Fold  1  - Acuracia:  0.814285714286
Fold  2  - Acuracia:  0.842857142857
Fold  3  - Acuracia:  0.814285714286
Fold  4  - Acuracia:  0.828571428571
Fold  5  - Acuracia:  0.8
Fold  6  - Acuracia:  0.8
Fold  7  - Acuracia:  0.785714285714
Fold  8  - Acuracia:  0.842857142857
Fold  9  - Acuracia:  0.8

Accuracia media:  0.814547283702
Desvio padrão:  0.0180871882636


SVM
Cross Validation:
Fold  0  - Acuracia:  0.774647887324
Fold  1  - Acuracia:  0.842857142857
Fold  2  - Acuracia:  0.8
Fold  3  - Acuracia:  0.814285714286
Fold  4  - Acuracia:  0.871428571429
Fold  5  - Acuracia:  0.828571428571
Fold  6  - Acuracia:  0.814285714286
Fold  7  - Acuracia:  0.714285714286
Fold  8  - Acuracia:  0.728571428571
Fold  9  - Acuracia:  0.728571428571

Accuracia media:  0.791750503018
Desvio padrão:  0.0506719580507

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.816901408451
Fold  1  - Acuracia:  0.857142857143
Fold  2  - Acura

In [17]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(1,2))

neg_counts = cv.fit_transform(neg_train["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_train["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}


In [19]:
from pandas import Series
s = Series(neg_excl_bigrams)
s.nlargest(15)

minas energia           15
edison                  12
edison lobão            12
sabia                   12
alves                   11
sabesp                  11
alves pmdb              10
câmara                  10
eduardo alves           10
henrique eduardo        10
morto                   10
presidente república    10
vaccari                 10
calheiros                9
envolvidos esquema       9
dtype: int64

In [19]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(1,1), stop_words= stopwords_pt, vocabulary  = lexicon)),
                    ("lexicon_vector", CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Fold  0  - Acuracia:  0.87323943662
Fold  1  - Acuracia:  0.828571428571
Fold  2  - Acuracia:  0.871428571429
Fold  3  - Acuracia:  0.885714285714
Fold  4  - Acuracia:  0.914285714286
Fold  5  - Acuracia:  0.828571428571
Fold  6  - Acuracia:  0.857142857143
Fold  7  - Acuracia:  0.871428571429
Fold  8  - Acuracia:  0.857142857143
Fold  9  - Acuracia:  0.814285714286

Accuracia media:  0.860181086519
Desvio padrão:  0.028506096132


SVM
Cross Validation:
Fold  0  - Acuracia:  0.633802816901
Fold  1  - Acuracia:  0.728571428571
Fold  2  - Acuracia:  0.671428571429
Fold  3  - Acuracia:  0.714285714286
Fold  4  - Acuracia:  0.714285714286
Fold  5  - Acuracia:  0.685714285714
Fold  6  - Acuracia:  0.671428571429
Fold  7  - Acuracia:  0.657142857143
Fold  8  - Acuracia:  0.628571428571
Fold  9  - Acuracia:  0.7

Accuracia media:  0.680523138833
Desvio padrão:  0.0325401073154

Maxent
Cross Validation:
Fold  0  - Acuracia:  0.887323943662
Fold  1  - Acuracia:  